In [ ]:

import pandas as pd
import matplotlib.pyplot as plt

from datetime import datetime, timedelta, date

from quantopian.pipeline import Pipeline
from quantopian.pipeline.data import Fundamentals as F
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.pipeline.filters import StaticAssets
from quantopian.research import run_pipeline


BILLION = 1000000000

MARKER_INDEX = 0
COLOR_INDEX = 1
DASH_INDEX = 2

MAIN = 1
RATIO = 2
OTHER = 3
CLOSE = 4

ATTR = {
    'close': ('.', 'k', tuple()),
    'market_cap': ('.', 'k', tuple()),
    'revenue': ('s', 'orange', (2,2)),
    'net_income': ('o', 'b', (2,2)),
    'fcf': ('^', 'g', (2,2)),
    'equity': ('*', 'grey', (2,2)),
    
    'ps': ('s', 'orange', (2,2)),
    'pe': ('o', 'b', (2,2)),
    'pfcf': ('^', 'g', (2,2)),
    'pb': ('*', 'grey', (2,2)),
    
    'revenue_growth': ('p', 'sandybrown', (2,2)),
    'gross_margin': ('+', 'gold', (2,2)),
    'rnd' : ('x', 'magenta', (2,2)),
    'op_margin': ('v', 'lightblue', (2,2)),
    'runway_util': ('>', 'darkgrey', (2,2)),
    'interest_util': ('<', 'dimgrey', (2,2)),
}


def make_pipeline(tickers):
    mask = StaticAssets(tickers)
    
    close = USEquityPricing.close.latest
    market_cap = F.market_cap.latest
    
    # big 4
    revenue = F.total_revenue.latest
    net_income = F.net_income_common_stockholders.latest
    fcf = F.free_cash_flow.latest
    equity = F.stockholders_equity.latest
    
    # ratios
    ps = F.ps_ratio.latest
    pe = F.pe_ratio.latest
    pfcf = F.fcf_ratio.latest
    pb = F.pb_ratio.latest
    
    # tech
    revenue_growth = F.revenue_growth.latest
    gross_margin = F.gross_margin.latest
    rnd = F.research_and_development.latest / revenue
    op_margin = F.operation_margin.latest
    runway_util = F.operating_expense.latest / F.cash_and_cash_equivalents.latest
    interest_util = F.interest_expense.latest / F.ebit.latest
    
    period_ending_date = F.period_ending_date.latest
    
    pipe = Pipeline(
        columns={
            'close': close,
            'period_ending_date': period_ending_date,
            'market_cap': market_cap,
            'revenue': revenue,
            'net_income': net_income,
            'fcf': fcf,
            'equity': equity,
            
            'ps': ps,
            'pe': pe,
            'pfcf': pfcf,
            'pb': pb,
            
            'revenue_growth': revenue_growth,
            'gross_margin': gross_margin,
            'rnd' : rnd,
            'op_margin': op_margin,
            'runway_util' : runway_util,
            'interest_util': interest_util,
        },
        screen=mask
    )
    return pipe


def prune_pipeline(pipeline, ticker):
    
    period_ending_dates = pd.to_datetime(pd.unique(pipeline['period_ending_date']))
    other_dates = period_ending_dates + timedelta(days=3)
    market_dates = period_ending_dates.union(other_dates)

    is_market_date = [True] * len(pipeline.index) #pipeline.index.get_level_values(0).isin(market_dates)
    is_ticker = pipeline.index.get_level_values(1) == symbols(ticker).sid
    return pipeline[is_market_date & is_ticker].reset_index()


def graph(pipeline, ax, columns, state):
    # TODO: generalize by passing in columns to graph and normalize boolean
    #  and randomly picking color/marker/shape
    
    def normalize_pipeline(col):
        denom = pipeline.loc[0, col]
        if state in (OTHER,RATIO):
            denom = 1
        if col == 'close': 
            denom = pipeline.loc[0, col]
        return pipeline[col] / denom
        #return BILLION
         
    x = pipeline['level_0']
    
    for column in columns:
        y = normalize_pipeline(column)
        ax.plot(x, y, 
                marker=ATTR[column][MARKER_INDEX],
                color=ATTR[column][COLOR_INDEX],
                dashes=ATTR[column][DASH_INDEX])
    ax.legend(columns, loc='upper left')   

    
TICKERS = symbols([

    #'aapl',
    #'msft',
    #'goog',
    #'amzn',
    #'dis',
    #'t',
    #'axp',
    #'v',
    'sq',
    #'pypl',
    #'jpm',
    
    #'eb',
    #'snap',
    #'docu',
    #'tdoc'
    #'m',
    #'bset',
    
    #'spot',

    #'plnhf',
    #'tcnnf'
])


def get_columns(state):
    columns = []
    
    if state == CLOSE:
        columns += [
            'close',
            #'market_cap',
        ]

    if state == MAIN:
        columns += [
            'revenue',
            'net_income',
            #'fcf',
            #'equity', 
        ]

    if state == RATIO:
        columns += [
            'ps',
            'pe',
            #'pfcf',
            #'pb',
        ]

    if state == OTHER:
        columns += [
        'revenue_growth',
        'gross_margin',
        #'rnd',
        'op_margin',
        #'runway_util',
        #'interest_util',    
        ]
    return columns
    
    
FIG_WIDTH = 15
FIG_HEIGHT = len(TICKERS)*30

debug = True
#state = MAIN # RATIO
states = (CLOSE, MAIN, RATIO, OTHER)
    
start_date = '2019-01-01'
yest = (datetime.now() - timedelta(days=1)).date()
_pipeline = run_pipeline(make_pipeline(TICKERS), start_date, yest)

fig, axs = plt.subplots(len(TICKERS)*len(states), 1, figsize=(FIG_WIDTH, FIG_HEIGHT))
plt.subplots_adjust(hspace=.5)

for i, ticker in enumerate(TICKERS):
    pipeline = prune_pipeline(_pipeline, ticker)
    
    for j, state in enumerate(states):
        index = len(states)*i+j
        axs[index].set_title(ticker)
        try:
            graph(pipeline, axs[index], get_columns(state), state)
        except Exception as e:
            print(ticker)        
            print('ERROR: ', e)
            print(pipeline.tail(1))
            if debug:
                raise e
            continue


# 